In [6]:
import numpy as np
from typing import List

class LifiAccessPoint:
    def __init__(self, x, y, Φ_1by2=60, Apd=100*1e-4, ref_index=1.5, FOV=90, gf=1,
            room_x=5, room_y=5, room_z=5, h=0.8, Rpd=0.53, pw=0.8, Popt=3, k=3, Nlifi=1e-21, Blifi=20*1e6):
        # attribute for Half-intensity radiation angle (Φ1/2)
        self.Φ_1by2 = Φ_1by2 * np.pi / 180
        self.m = -np.log(2) / np.log(np.cos(np.radians(Φ_1by2)))
        # Refractive ref_index of the medium
        self.ref_index = ref_index
        # FoV semi-angle of PD, Ψmax
        self.FOV = FOV * np.pi / 180
        # physical area of the PD, Apd
        self.Apd = Apd
        # gain of the optical filter, gf
        self.gf = gf
        # length of the room in the x-direction (horizontal) in meters.
        self.room_x = room_x
        # length of the room in the y-direction (vertical) in meters.
        self.room_y = room_y
        # height of the room in meters
        self.room_z = room_z
        # height of the user above the receiver plane
        self.h = h
        # position of the lifi access point in ceiling
        self.lifi_position = np.array([x, y, room_z])
        # Detector responsivity, Rpd
        self.Rpd = Rpd
        # Transmitted optical power per LiFi AP, Popt
        self.Popt = Popt
        # optical to electrical conversion coefficient, k
        self.k = k
        # PSD of noise in LiFi AP, NLiFi
        self.Nlifi = Nlifi
        # Bandwidth of LiFi AP, BLiFi
        self.Blifi = Blifi
        # Wall reflectivity, pw
        self.pw = pw
        
        # Number of points in the x and y directions
        # self.Nx = int(room_x * 10)
        # self.Ny = int(room_y * 10)
        # self.x = np.linspace(0, room_x, self.Nx)
        # self.y = np.linspace(0, room_y, self.Ny)
        # self.XR, self.YR = np.meshgrid(self.x, self.y)
        # print(self.XR)
        # print(self.XR.shape)

    def get_channel_gain(self, user_x, user_y):
        # uncomment this line to include NLOS channel gain
        return self.channel_gain_los(user_x, user_y) #+ self.channel_gain_nlos(user_x, user_y)

    def channel_gain_los(self, user_x, user_y):
        d = self.distance(user_x, user_y)        
        # both angles are equal due to symmetry
        incidence = self.angle_incidence(user_x, user_y)
        irradiance = incidence           
        gc = self.optical_gain(incidence)
        channel_gain = ((self.m + 1) * self.Apd * (np.cos(irradiance)**self.m) * np.cos(incidence) * self.gf * gc) / (2 * np.pi * (d**2))
        # print(incidence, irradiance, channel_gain, self.m, gc, self.Apd, self.gf)
        return channel_gain
    
    def channel_gain_nlos(self, user_x, user_y):
        boxes_per_meter = 10
        dl, dh = 1/boxes_per_meter, 1/boxes_per_meter
        height_coord = np.arange(self.h, self.room_z, dh)
        length_coord = np.arange(self.h, self.room_x, dl)
        user_position = np.array([user_x, user_y, self.h])
        integration = 0
        
        for height in height_coord:
            # print(f'{height:.3f}  ')
            for length in length_coord:
                locations = [np.array([length, 0, height]), np.array([length, self.room_y, height]),
                            np.array([0, length, height]), np.array([self.room_x, length, height])]
                for curr_location in locations:
                    d_iw = np.linalg.norm(curr_location - self.lifi_position)
                    d_wu = np.linalg.norm(user_position - curr_location)
                    theta_iw = np.arccos((self.room_z - height) / d_iw)
                    ϑ_iw = 90 - theta_iw
                    # print(f'd_iw: {d_iw}, d_wu: {d_wu}, theta_iw: {theta_iw}, ϑ_iw: {ϑ_iw}')
                    phi_wu = np.arccos((height - self.h) / d_wu)
                    ϑ_wu = 90 - phi_wu
                    numerator = (self.m + 1) * self.Apd * self.pw * (np.cos(theta_iw)**self.m) * \
                                self.gf * self.optical_gain(phi_wu) * np.cos(phi_wu) * np.cos(ϑ_iw) * np.cos(ϑ_wu)
                    integration += (numerator * dl * dh) / (2 * (np.pi * d_iw * d_wu)**2)
        return integration
    
    def signal_to_noise_ratio(self, user_x, user_y, otherLifiAPs:List):
        summation_term = 0
        # for lifi in otherLifiAPs:
        #     # print(f'Lifi at {lifi.lifi_position} has gain', lifi.get_channel_gain(user_x, user_y))
        #     summation_term += (lifi.Rpd * lifi.get_channel_gain(user_x, user_y) * lifi.Popt / lifi.k) ** 2
        # print('self gain: ', self.get_channel_gain(user_x, user_y))
        numerator = (self.Rpd * self.get_channel_gain(user_x, user_y) * self.Popt / self.k) ** 2
        # uncomment this line to include noise from other LiFi APs
        denominator = self.Nlifi * self.Blifi + summation_term 
        # print(f'numerator: {numerator}, denominator: {denominator}, sum: {summation_term}')   
        return numerator / denominator
    
    def distance(self, user_x, user_y):
        user_position = np.array([user_x, user_y, self.h])
        distance =  np.linalg.norm(user_position - self.lifi_position)
        return distance
    
    def optical_gain(self, angle_incidence):
        if 0 <= angle_incidence <= self.FOV:
            return (self.ref_index**2) / (np.sin(self.FOV)**2)
        else:
            return 0
    
    def angle_incidence(self, user_x, user_y):
        d = self.distance(user_x, user_y)
        return np.arccos((self.room_z - self.h) / d)





In [7]:
# if __name__ == "__main__":
#     x, y = 1.25, 1.25
#     lifi_ap1 = LifiAccessPoint(x=1.25, y=1.25)
#     # lifi_ap2 = LifiAccessPoint(x=1.25, y=3.75)
#     # lifi_ap3 = LifiAccessPoint(x=3.75, y=3.75)
#     # lifi_ap4 = LifiAccessPoint(x=3.75, y=1.25)

#     # ang_incidence = lifi_ap1.angle_incidence(x, y)
#     # optical_gain = lifi_ap1.optical_gain(ang_incidence)
#     # # print(ang_incidence)
#     # # print(optical_gain)
#     # # H = lifi_ap1.get_channel_gain(x, y)
    
#     # print('Location: ', 0, 0)
#     # snr = lifi_ap1.signal_to_noise_ratio(0, 0, otherLifiAPs=[lifi_ap2, lifi_ap3, lifi_ap4])
#     # print(snr)
#     # print('Location: ', 1.25, 1.25)
#     # snr = lifi_ap1.signal_to_noise_ratio(1.2, 1.2, otherLifiAPs=[lifi_ap2, lifi_ap3, lifi_ap4])
#     # print(snr)

In [8]:
x, y = 1.25, 1.25
lifi_ap1 = LifiAccessPoint(x=1.25, y=1.25)

In [9]:
room_width = 5.0
room_height = 5.0
user_height = 0.8

# Divide the floor into 0.1x0.1m squares
grid_size = 0.1
# x_grid = np.arange(0, room_width, grid_size)
# y_grid = np.arange(0, room_height, grid_size)
x_grid = [round(i * grid_size, 1) for i in range(int(room_width / grid_size) + 1)]
y_grid = [round(i * grid_size, 1) for i in range(int(room_height / grid_size) + 1)]

print(f"Number of squares in x-direction: {len(x_grid)}")
print(f"Number of squares in y-direction: {len(y_grid)}")
print(f"Total number of squares: {len(x_grid) * len(y_grid)}")
print()
print("Grid points:")
# print(list(zip(x_grid, y_grid)))
# make list of all ombination of x and y
grid_points = [(x, y) for x in x_grid for y in y_grid]
print(grid_points)

Number of squares in x-direction: 51
Number of squares in y-direction: 51
Total number of squares: 2601

Grid points:
[(0.0, 0.0), (0.0, 0.1), (0.0, 0.2), (0.0, 0.3), (0.0, 0.4), (0.0, 0.5), (0.0, 0.6), (0.0, 0.7), (0.0, 0.8), (0.0, 0.9), (0.0, 1.0), (0.0, 1.1), (0.0, 1.2), (0.0, 1.3), (0.0, 1.4), (0.0, 1.5), (0.0, 1.6), (0.0, 1.7), (0.0, 1.8), (0.0, 1.9), (0.0, 2.0), (0.0, 2.1), (0.0, 2.2), (0.0, 2.3), (0.0, 2.4), (0.0, 2.5), (0.0, 2.6), (0.0, 2.7), (0.0, 2.8), (0.0, 2.9), (0.0, 3.0), (0.0, 3.1), (0.0, 3.2), (0.0, 3.3), (0.0, 3.4), (0.0, 3.5), (0.0, 3.6), (0.0, 3.7), (0.0, 3.8), (0.0, 3.9), (0.0, 4.0), (0.0, 4.1), (0.0, 4.2), (0.0, 4.3), (0.0, 4.4), (0.0, 4.5), (0.0, 4.6), (0.0, 4.7), (0.0, 4.8), (0.0, 4.9), (0.0, 5.0), (0.1, 0.0), (0.1, 0.1), (0.1, 0.2), (0.1, 0.3), (0.1, 0.4), (0.1, 0.5), (0.1, 0.6), (0.1, 0.7), (0.1, 0.8), (0.1, 0.9), (0.1, 1.0), (0.1, 1.1), (0.1, 1.2), (0.1, 1.3), (0.1, 1.4), (0.1, 1.5), (0.1, 1.6), (0.1, 1.7), (0.1, 1.8), (0.1, 1.9), (0.1, 2.0), (0.1, 2.1), (0.1,

In [10]:
d = {}
for x, y in grid_points:
    d[(x, y)] = lifi_ap1.channel_gain_nlos(x, y)

C:\Users\sumit\AppData\Local\Temp\ipykernel_1284\2975731896.py:83: RuntimeWarning: invalid value encountered in double_scalars
  phi_wu = np.arccos((height - self.h) / d_wu)


In [12]:
for i,j in d.items():
    #if j=nan then assign 0
    if np.isnan(j):
        d[i]=0

In [13]:
d

{(0.0, 0.0): 5.4234414059074406e-06,
 (0.0, 0.1): 4.50563657738091e-06,
 (0.0, 0.2): 3.535245400513943e-06,
 (0.0, 0.3): 2.487800296105182e-06,
 (0.0, 0.4): 1.3287029179364552e-06,
 (0.0, 0.5): 9.264983400362365e-09,
 (0.0, 0.6): -1.526489339430234e-06,
 (0.0, 0.7): -2.7889118195212678e-06,
 (0.0, 0.8): 0,
 (0.0, 0.9): 0,
 (0.0, 1.0): 0,
 (0.0, 1.1): 0,
 (0.0, 1.2): 0,
 (0.0, 1.3): -176055658.48356807,
 (0.0, 1.4): 0,
 (0.0, 1.5): 0,
 (0.0, 1.6): -164291830.29374605,
 (0.0, 1.7): -156790179.0684096,
 (0.0, 1.8): -147768806.04070726,
 (0.0, 1.9): -137429076.2760459,
 (0.0, 2.0): -125984008.85644765,
 (0.0, 2.1): -28412348.300684314,
 (0.0, 2.2): -25159151.768095143,
 (0.0, 2.3): 0,
 (0.0, 2.4): -18342264.82327672,
 (0.0, 2.5): 0,
 (0.0, 2.6): -11404070.931806264,
 (0.0, 2.7): -1995761.4619496362,
 (0.0, 2.8): 0,
 (0.0, 2.9): -1385047.414188833,
 (0.0, 3.0): 436438.40292589634,
 (0.0, 3.1): 4740354.139897606,
 (0.0, 3.2): 1896330.3218183478,
 (0.0, 3.3): 0,
 (0.0, 3.4): 12788935.54724959

In [17]:
import json
with open('channel_gain_nlos.json', 'w') as fp:
    json.dump(d, fp, indent=4)

In [15]:
# change the keys to string
d = {str(k): v for k, v in d.items()}

In [16]:
d

{'(0.0, 0.0)': 5.4234414059074406e-06,
 '(0.0, 0.1)': 4.50563657738091e-06,
 '(0.0, 0.2)': 3.535245400513943e-06,
 '(0.0, 0.3)': 2.487800296105182e-06,
 '(0.0, 0.4)': 1.3287029179364552e-06,
 '(0.0, 0.5)': 9.264983400362365e-09,
 '(0.0, 0.6)': -1.526489339430234e-06,
 '(0.0, 0.7)': -2.7889118195212678e-06,
 '(0.0, 0.8)': 0,
 '(0.0, 0.9)': 0,
 '(0.0, 1.0)': 0,
 '(0.0, 1.1)': 0,
 '(0.0, 1.2)': 0,
 '(0.0, 1.3)': -176055658.48356807,
 '(0.0, 1.4)': 0,
 '(0.0, 1.5)': 0,
 '(0.0, 1.6)': -164291830.29374605,
 '(0.0, 1.7)': -156790179.0684096,
 '(0.0, 1.8)': -147768806.04070726,
 '(0.0, 1.9)': -137429076.2760459,
 '(0.0, 2.0)': -125984008.85644765,
 '(0.0, 2.1)': -28412348.300684314,
 '(0.0, 2.2)': -25159151.768095143,
 '(0.0, 2.3)': 0,
 '(0.0, 2.4)': -18342264.82327672,
 '(0.0, 2.5)': 0,
 '(0.0, 2.6)': -11404070.931806264,
 '(0.0, 2.7)': -1995761.4619496362,
 '(0.0, 2.8)': 0,
 '(0.0, 2.9)': -1385047.414188833,
 '(0.0, 3.0)': 436438.40292589634,
 '(0.0, 3.1)': 4740354.139897606,
 '(0.0, 3.2)': 